# Prediction and Evaluation

This notebook handles model prediction and evaluation using the trained model.
It includes:
- Loading trained model
- Making predictions
- Model evaluation
- Visualization of results
- Error analysis

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from datetime import datetime

# Add project root to path
sys.path.append('..')

# Import project modules
from src.components.model import CloudNowcastingModel
from src.utils import evaluate_model
from src.logger import logger
from src.exception import CustomException

# Configure logging
logger.info("Starting prediction and evaluation notebook")

## 1. Load Test Data and Model

In [ ]:
def load_test_data_and_model(data_dir='artifacts', model_dir='artifacts'):
    """
    Load test data and trained model
    
    Args:
        data_dir (str): Directory containing test data
        model_dir (str): Directory containing trained model
    
    Returns:
        tuple: X_test, y_test, loaded model
    """
    try:
        # Load test data
        X_test = np.load(os.path.join(data_dir, 'X_test.npy'))
        y_test = np.load(os.path.join(data_dir, 'y_test.npy'))
        
        # Load trained model
        model_path = os.path.join(model_dir, 'best_model.h5')
        model = tf.keras.models.load_model(model_path)
        
        logger.info("Test data and model loaded successfully")
        return X_test, y_test, model
    
    except Exception as e:
        logger.error(f"Failed to load test data or model: {e}")
        raise CustomException(e, sys)

# Load data and model
X_test, y_test, model = load_test_data_and_model()

print("Data shapes:")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

## 2. Make Predictions

In [ ]:
def make_predictions(model, X_test):
    """
    Generate predictions using the trained model
    
    Args:
        model (tf.keras.Model): Trained model
        X_test (np.ndarray): Test input data
    
    Returns:
        tuple: Predictions and prediction uncertainties
    """
    try:
        # Initialize model for prediction
        model_trainer = CloudNowcastingModel()
        
        # Make predictions
        predictions, prediction_std = model_trainer.predict(model, X_test)
        
        logger.info("Predictions generated successfully")
        return predictions, prediction_std
    
    except Exception as e:
        logger.error(f"Prediction generation failed: {e}")
        raise CustomException(e, sys)

# Generate predictions
predictions, prediction_std = make_predictions(model, X_test)

print("Predictions completed")
print(f"Predictions shape: {predictions.shape}")
print(f"Prediction uncertainty shape: {prediction_std.shape}")

## 3. Model Evaluation

In [ ]:
def evaluate_model_performance(y_test, predictions):
    """
    Evaluate model performance using various metrics
    
    Args:
        y_test (np.ndarray): True test labels
        predictions (np.ndarray): Model predictions
    
    Returns:
        dict: Evaluation metrics
    """
    try:
        # Calculate evaluation metrics
        metrics = evaluate_model(y_test, predictions)
        
        logger.info("Model evaluation completed")
        return metrics
    
    except Exception as e:
        logger.error(f"Model evaluation failed: {e}")
        raise CustomException(e, sys)

# Evaluate model
metrics = evaluate_model_performance(y_test, predictions)

print("\nModel Evaluation Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

## 4. Visualize Predictions

In [ ]:
def plot_predictions(X, y_true, y_pred, output_dir=None, num_samples=3):
    """
    Visualize input sequences, true targets, and predictions
    
    Args:
        X (np.ndarray): Input sequences
        y_true (np.ndarray): True target values
        y_pred (np.ndarray): Predicted values
        output_dir (str, optional): Directory to save plot
        num_samples (int): Number of samples to plot
    """
    try:
        fig, axes = plt.subplots(num_samples, 3, figsize=(15, 4*num_samples))
        
        for i in range(num_samples):
            # Plot input sequence
            axes[i, 0].imshow(X[i, -1, :, :, 0], cmap='viridis')
            axes[i, 0].set_title(f'Input Sequence (t={i})')
            axes[i, 0].axis('off')
            
            # Plot true target
            axes[i, 1].imshow(y_true[i, 0, :, :, 0], cmap='viridis')
            axes[i, 1].set_title(f'True Target (t={i+1})')
            axes[i, 1].axis('off')
            
            # Plot prediction
            axes[i, 2].imshow(y_pred[i, 0, :, :, 0], cmap='viridis')
            axes[i, 2].set_title(f'Prediction (t={i+1})')
            axes[i, 2].axis('off')
        
        plt.tight_layout()
        
        # Save or show plot
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            plt.savefig(os.path.join(output_dir, 'prediction_comparison.png'))
            plt.close()
        else:
            plt.show()
        
        logger.info("Prediction visualization completed")
    
    except Exception as e:
        logger.error(f"Prediction visualization failed: {e}")
        raise CustomException(e, sys)

# Plot sample predictions
plot_predictions(X_test, y_test, predictions, output_dir='artifacts')

## 5. Error Analysis

In [ ]:
def analyze_prediction_errors(y_true, y_pred, prediction_std, output_dir=None):
    """
    Perform comprehensive error analysis
    
    Args:
        y_true (np.ndarray): True target values
        y_pred (np.ndarray): Predicted values
        prediction_std (np.ndarray): Prediction uncertainties
        output_dir (str, optional): Directory to save analysis plot
    
    Returns:
        dict: Error statistics
    """
    try:
        # Calculate absolute errors
        abs_errors = np.abs(y_true - y_pred)
        
        # Prepare plot
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Error distribution
        axes[0].hist(abs_errors.flatten(), bins=50, color='skyblue', edgecolor='black')
        axes[0].set_title('Distribution of Absolute Errors')
        axes[0].set_xlabel('Absolute Error')
        axes[0].set_ylabel('Frequency')
        
        # Error vs Uncertainty scatter
        scatter = axes[1].scatter(
            prediction_std.flatten(), 
            abs_errors.flatten(), 
            alpha=0.1, 
            c=prediction_std.flatten(), 
            cmap='viridis'
        )
        axes[1].set_title('Error vs Prediction Uncertainty')
        axes[1].set_xlabel('Prediction Uncertainty')
        axes[1].set_ylabel('Absolute Error')
        plt.colorbar(scatter, ax=axes[1], label='Uncertainty')
        
        plt.tight_layout()
        
        # Save or show plot
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            plt.savefig(os.path.join(output_dir, 'error_analysis.png'))
            plt.close()
        else:
            plt.show()
        
        # Compute error statistics
        error_stats = {
            'mean_absolute_error': np.mean(abs_errors),
            'median_absolute_error': np.median(abs_errors),
            'percentile_95_error': np.percentile(abs_errors, 95),
            'max_error': np.max(abs_errors),
            'error_std': np.std(abs_errors)
        }
        
        logger.info("Error analysis completed")
        return error_stats
    
    except Exception as e:
        logger.error(f"Error analysis failed: {e}")
        raise CustomException(e, sys)

# Analyze prediction errors
error_stats = analyze_prediction_errors(y_test, predictions, prediction_std, output_dir='artifacts')

print("\nError Statistics:")
for stat_name, value in error_stats.items():
    print(f"{stat_name.replace('_', ' ').title()}: {value:.4f}")

## 6. Save Evaluation Results

In [ ]:
def save_evaluation_results(metrics, error_stats, predictions, prediction_std, output_dir='artifacts'):
    """
    Save evaluation metrics, error statistics, and predictions
    
    Args:
        metrics (dict): Model evaluation metrics
        error_stats (dict): Error analysis statistics
        predictions (np.ndarray): Model predictions
        prediction_std (np.ndarray): Prediction uncertainties
        output_dir (str): Directory to save results
    """
    try:
        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        # Save evaluation metrics
        metrics_df = pd.DataFrame(metrics, index=[0])
        metrics_df.to_csv(os.path.join(output_dir, 'evaluation_metrics.csv'), index=False)
        
        # Save error statistics
        error_stats_df = pd.DataFrame([error_stats])
        error_stats_df.to_csv(os.path.join(output_dir, 'error_statistics.csv'), index=False)
        
        # Save predictions
        np.save(os.path.join(output_dir, 'predictions.npy'), predictions)
        np.save(os.path.join(output_dir, 'prediction_uncertainty.npy'), prediction_std)
        
        logger.info("Evaluation results saved successfully")
        print("Evaluation results saved in artifacts directory")
    
    except Exception as e:
        logger.error(f"Failed to save evaluation results: {e}")
        raise CustomException(e, sys)

# Save evaluation results
save_evaluation_results(metrics, error_stats, predictions, prediction_std)